
# Clasificación del tipo de día en España

Este cuaderno genera un DataFrame donde para cada día se añade información sobre el tipo de día, útil el para modelado de precios eléctricos:

- Tipo de día:
  - *F*: festivo nacional en España.
  - *FR*: festivo en al menos una comunidad autónoma.
  - *L*, *M*, *X*, *J*, *V*, *S*, *D*: días de la semana.

La detección de festivos se realiza con la librería `holidays`.

In [1]:
import pandas as pd
from datetime import date

# Configuración de fechas
start_date = date(2016, 1, 1)
end_date = date.today()

# Crear rango de fechas
all_days = pd.date_range(start=start_date, end=end_date, freq="D")

years = list(range(start_date.year, end_date.year + 1))


## Mapeo de días de la semana

Usamos la convención:
- L: Lunes
- M: Martes
- X: Miércoles
- J: Jueves
- V: Viernes
- S: Sábado
- D: Domingo


In [2]:
weekday_map = {0: "L", 1: "M", 2: "X", 3: "J", 4: "V", 5: "S", 6: "D"}


## Festivos nacionales y regionales



In [3]:
# Calcular fechas de Semana Santa (Jueves y Viernes Santo) para cada año
from dateutil.easter import easter

semana_santa = []
for y in years:
    jueves_santo = easter(y) - pd.Timedelta(days=2)
    viernes_santo = easter(y) - pd.Timedelta(days=1)
    semana_santa.extend([(jueves_santo.year, jueves_santo.month, jueves_santo.day), (viernes_santo.year, viernes_santo.month, viernes_santo.day)])

semana_santa

[(2016, 3, 25),
 (2016, 3, 26),
 (2017, 4, 14),
 (2017, 4, 15),
 (2018, 3, 30),
 (2018, 3, 31),
 (2019, 4, 19),
 (2019, 4, 20),
 (2020, 4, 10),
 (2020, 4, 11),
 (2021, 4, 2),
 (2021, 4, 3),
 (2022, 4, 15),
 (2022, 4, 16),
 (2023, 4, 7),
 (2023, 4, 8),
 (2024, 3, 29),
 (2024, 3, 30),
 (2025, 4, 18),
 (2025, 4, 19)]

In [4]:
# Crear DataFrame con fechas
df_dias = pd.DataFrame({"fecha": all_days})

# Lista de festivos nacionales 
festivos_nacionales = [
    (1, 1),   # Año Nuevo
    (1, 6),   # Reyes
    (5, 1),   # Día del Trabajador
    (8, 15),  # Asunción
    (10, 12), # Fiesta Nacional
    (11, 1),  # Todos los Santos
    (12, 6),  # Constitución
    (12, 8),  # Inmaculada
    (12, 25), # Navidad
]

# Lista de festivos regionales
festivos_regionales = {
    (2, 28),  # Día de Andalucía
    (3, 19),  # San José (Comunidad Valenciana, Murcia)
    (4, 23),  # Día de Castilla-León y San Jorge (Aragón, Cáceres)
    (5, 2),   # Día de Madrid
    (5, 31),  # Día de Castilla-La Mancha
    (6, 9),   # Día de la Región de Murcia y de La Rioja
    (7, 25),  # Día de Galicia
    (7, 28),  # Día de Cantabria
    (9, 8),   # Día de Asturias y Extremadura
    (9, 11),  # Día de Cataluña
    (10, 9),  # Día de la Comunidad Valenciana
    (10, 25), # Día del País Vasco
    (12, 3),  # Día de Navarra
}


def tipo_dia_func(fecha):
    d = fecha.date()
    if (d.month, d.day) in festivos_nacionales:
        return "F"
    if (d.year, d.month, d.day) in semana_santa:
        return "F"
    elif (d.month, d.day) in festivos_regionales:
        return "FR"  
    return weekday_map[d.weekday()]

df_dias["tipo_dia"] = df_dias["fecha"].apply(tipo_dia_func)
df_dias

,fecha,tipo_dia
0,2016-01-01,F
1,2016-01-02,S
2,2016-01-03,D
3,2016-01-04,L
4,2016-01-05,M
...,...,...
3520,2025-08-21,J
3521,2025-08-22,V
3522,2025-08-23,S
3523,2025-08-24,D



## Exportación

Se guarda el DataFrame en CSV.


In [8]:
csv_path = "../datos/tipo_dias.csv"
df_dias.to_csv(csv_path, index=False)
print("CSV exportado a:", csv_path)

CSV exportado a: ../datos/tipo_dias.csv
